## Connect to the NHANES data resource using the HPDS Adapter

In [38]:
library(PicSureClient)

adapter <- PicSureHpdsLib::BypassAdapter("http://pic-sure-hpds-nhanes:8080/PIC-SURE")
resource <- adapter$useResource()

### A "resource" allows access to the Data Dictionary and Query Engine of the underlying  data source

In [30]:
resource$help()


        [HELP] PicSureHpdsLib.useResource(resource_uuid)
            .dictionary()       Used to access data dictionary of the resource
            .query()            Used to query against data in the resource
            
        [ENVIRONMENT]
              Endpoint URL: http://pic-sure-hpds-nhanes:8080/PIC-SURE/
             Resource UUID: None


## Create a new query instance and use .help() to see what can be done with it

In [3]:
query <- resource$query()
query$help()


        .select()   list of data fields to return from resource for each record
        .require()  list of data fields that must be present in all returned records
        .filter()   list of data fields and conditions that returned records satisfy
                  [ Filter keys exert an AND relationship on returned records      ]
                  [ Categorical values have an OR relationship on their key        ]
                  [ Numerical Ranges are inclusive of their start and end points   ]

        .getCount()             returns a count indicating the number of matching numbers
        .getResults()           returns a CSV-like string containing the matching records
        .getResultsDataFrame()  returns a pandas DataFrame containing the matching records
        .getRunDetails()        returns details about the last run of the query
        .getQueryCommand()      returns the JSON-formatted query request
        .show()                 lists all current query parameters
  

### Use .help() to see how to add entries to the selection criteria

In [5]:
query$select()$help()


            select().
              add("key")            add a single column to be returned in results
              add(["key1", "key2"]) add several columns to be returned in results
              delete("key")         delete a single column from the list of columns to return
              show()                lists all current columns that will be returned in results
              clear()               clears all values from the select list
            


In [ ]:
### Use a chained of commands on the dictionary object to include all labs in the query's selection list

In [11]:
query$select()$add(resource$dictionary()$find("laboratory")$keys())

### Use the .show() command to see what is currently selected for the query

In [12]:
query$show()

.__________[ Query.Select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\demographics\\AGE\\                                                                                                            |
|  \\laboratory\\volatile compounds\\Blood MTBE (pg per mL)\\                                                                       |
|  \\laboratory\\perchlorate\\Perchlorate, water (ng per mL)\\                                                                      |
|  \\laboratory\\volatile compounds\\Blood 1,2-Dichloroethane (ng per mL)\\                                                         |
|  \\laboratory\\volatile compounds\\Toluene (ug per cubic meter)\\                                                                 |
|  \\laboratory\\polyflourochemicals\\Perflu

### Clear the current selection list and add only "\\laboratory\\hormone\\" results

In [46]:
query$select()$clear()
query$select()$add(resource$dictionary()$find("\\laboratory\\hormone\\")$keys())
query$select()$show()

cleared list
| _key__________________________________________________________________________________________________________________________
|  \\laboratory\\hormone\\Insulin: SI(pmol per L)\\                                                                                 |
|  \\laboratory\\hormone\\Thyroxine (T4) (ug per dL)\\                                                                              |
|  \\laboratory\\hormone\\Luteinizing hormone (mIU per mL)\\                                                                        |
|  \\laboratory\\hormone\\Thyroid stim hormone (TSH) (IU per L)\\                                                                   |
|  \\laboratory\\hormone\\Parathyroid Hormone(Elecys method) pg per mL\\                                                            |
|  \\laboratory\\hormone\\Insulin (uU per mL)\\                                                                                     |
|  \\laboratory\\hormone\\                            

In [ ]:
### Run some commands on the dictionary to find and then filter the query by gender 

In [47]:
gender <- resource$dictionary()$find("sex")
gender$help()


        [HELP] PicSureHpdsLib.Client(connection).useResource(uuid).dictionary().find(term)
            .count()        Returns the number of entries in the dictionary that match the given term
            .keys()         Return the keys of the matching entries
            .entries()      Return a list of matching dictionary entries
            .DataFrame()    Return the entries in a Pandas-compatible format
             
        [Examples]
            results = PicSureHpdsLib.Client(connection).useResource(uuid).dictionary().find("asthma")
            df = results.DataFrame()
        


In [48]:
gender$DataFrame()

,categorical,categoryValues,observationCount,min,max
\questionnaire\sexual behavior\Are you circumcised or uncircumcised\,True,"[No, Yes]",5178,NaN,NaN
\demographics\SEX\,True,"[female, male]",41474,NaN,NaN
\questionnaire\sexual behavior\Ever had sexual intercourse\,True,"[No, Yes]",8271,NaN,NaN
\questionnaire\sexual behavior\,False,None,41474,0.0,17040.0


### Add a filter on our query to filter results to only female subjects

In [23]:
query$filter()$help()


            filter().
              add("key", value)                  - or -
              add("key", "value")               filter to records with KEY column that equals VALUE
              add("key", ["value1", "value2"])  filter to records with KEY column equalling one value within the given list
              add("key", start, end)            filter to records with KEY column value between START and END (inclusive)
                                                    start -or- end may be set to None to filter by a max or min value
              delete("key")                     delete a filter from the list of filters
              show()                            lists all current filters that results records must satisfy
              clear()                           clears all values from the filters list
            


In [49]:
query$filter()$add("\\demographics\\SEX\\", list("female"))

ERROR: cannot add, key already exists ->  \demographics\SEX\


## View our query's criterion as it currently stands

In [25]:
query$show()

.__________[ Query.Select()  Settings ]_____________________________________________________________________________________________________________________
| _key__________________________________________________________________________________________________________________________
|  \\laboratory\\hormone\\Insulin: SI(pmol per L)\\                                                                                 |
|  \\laboratory\\hormone\\Thyroxine (T4) (ug per dL)\\                                                                              |
|  \\laboratory\\hormone\\Luteinizing hormone (mIU per mL)\\                                                                        |
|  \\laboratory\\hormone\\Thyroid stim hormone (TSH) (IU per L)\\                                                                   |
|  \\laboratory\\hormone\\Parathyroid Hormone(Elecys method) pg per mL\\                                                            |
|  \\laboratory\\hormone\\Insulin (uU per mL

### Run the query and display the results

In [26]:
query$help()


        .select()   list of data fields to return from resource for each record
        .require()  list of data fields that must be present in all returned records
        .filter()   list of data fields and conditions that returned records satisfy
                  [ Filter keys exert an AND relationship on returned records      ]
                  [ Categorical values have an OR relationship on their key        ]
                  [ Numerical Ranges are inclusive of their start and end points   ]

        .getCount()             returns a count indicating the number of matching numbers
        .getResults()           returns a CSV-like string containing the matching records
        .getResultsDataFrame()  returns a pandas DataFrame containing the matching records
        .getRunDetails()        returns details about the last run of the query
        .getQueryCommand()      returns the JSON-formatted query request
        .show()                 lists all current query parameters
  

In [50]:
query$getCount()

21210

In [51]:
query$getResultsDataFrame()

,Patient ID,\demographics\SEX\,\laboratory\hormone\,\laboratory\hormone\Follicle stimulating hormone (mIU per mL)\,\laboratory\hormone\Insulin (uU per mL)\,\laboratory\hormone\Insulin: SI(pmol per L)\,\laboratory\hormone\Luteinizing hormone (mIU per mL)\,\laboratory\hormone\Parathyroid Hormone(Elecys method) pg per mL\,\laboratory\hormone\Thyroid stim hormone (TSH) (IU per L)\,\laboratory\hormone\Thyroxine (T4) (ug per dL)\
0,3,female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,female,NaN,46.10,NaN,NaN,32.44,NaN,0.85,7.8
2,6,female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,female,NaN,25.99,8.30,49.80,8.65,NaN,2.50,7.8
4,11,female,NaN,NaN,NaN,NaN,NaN,NaN,0.91,9.4
5,12,female,NaN,NaN,7.04,42.24,NaN,NaN,NaN,NaN
6,13,female,0.90,NaN,12.92,77.52,NaN,NaN,NaN,NaN
7,14,female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15,female,NaN,NaN,29.50,NaN,NaN,56.0,NaN,NaN
9,16,female,NaN,NaN,15.73,94.38,NaN,NaN,1.88,8.3


# You can even work with multiple queries at the same time without having them accidently interact with each other!

In [43]:
query2 <- resource$query()
query3 <- resource$query()


query2$select()$add(resource.dictionary()$find("\\demographics\\AGE")$keys())
query3$select()$add(resource.dictionary()$find("\\demographics\\AGE")$keys())


#query2$filter()$add(resource$dictionary()$find("\\demographics\\AGE").keys(), min=60)
#query3$filter()$add(resource$dictionary()$find("\\demographics\\AGE").keys(), max=20)
query2$filter()$add(resource$dictionary()$find("\\demographics\\AGE")$keys(), 60, NULL)
query3$filter()$add(resource$dictionary()$find("\\demographics\\AGE")$keys(), NULL, 20)

too_old <- query2$getResultsDataFrame()
too_young <- query3$getResultsDataFrame()

In [44]:
too_young

,Patient ID,\demographics\AGE\
0,1,45.0
1,2,85.0
2,3,0.0
3,4,49.0
4,5,18.0
5,6,4.0
6,7,31.0
7,8,40.0
8,9,52.0
9,10,25.0


In [45]:
too_old

,Patient ID,\demographics\AGE\
